---
### 📦 Información de Versionado con DVC y MLflow

**Este notebook es el PASO 3 del pipeline:**
1. Lee: `data/processed/student_performance_features.csv` (versión con features - `data-v1.2-features`)
2. Entrena: Modelos LightGBM, XGBoost y CatBoost
3. Registra: Experimentos y métricas con MLflow
4. Guarda: Modelos entrenados (versionados con DVC)

**⚠️ Prerrequisitos:**
1. ✅ Haber ejecutado: `1_EDA_and_Cleaning.ipynb`
2. ✅ Haber ejecutado: `2_Data_Processing.ipynb`
3. ✅ Haber versionado los resultados con DVC

**Para obtener el dataset correcto:**
```bash
# Si tu compañero ya lo procesó, descarga la versión con features
dvc pull

# O asegúrate de estar en la versión correcta
git checkout data-v1.2-features
dvc checkout
```

**Versionado de modelos:**
Este notebook usa MLflow para trackear experimentos, pero los modelos finales también se versionan con DVC para tener un control completo del pipeline.

---


# 1. Validación del Dataset Procesado (data-v1.2-features)

El objetivo de esta sección dentro de la fase 2 es: 

- Validar que el dataset de características finales (student_performance_features.csv) esté correcto antes de entrenar los modelos:
- Verificar que las features (PC1–PC15) y la etiqueta (Performance) sean coherentes.
- Confirmar que los datos estén listos para entrar al pipeline de modelado sin errores ni inconsistencias.

In [20]:
import pandas as pd
import numpy as np
import mlflow
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.metrics import cohen_kappa_score, root_mean_squared_error
from sklearn.model_selection import train_test_split

In [27]:
df = pd.read_csv("../data/processed/student_performance_features.csv")
df

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,Performance
0,0.803234,-0.597979,-0.285577,-0.770143,-0.017667,0.023372,-0.063243,-0.451023,-0.984059,-0.048342,-0.116927,0.258324,-0.415056,-0.299239,-0.045746,3
1,0.383741,-0.558693,0.664729,1.081237,1.002544,-0.000102,0.260944,0.398450,-0.258088,0.143168,0.352092,0.023940,0.116888,0.069941,-0.081184,3
2,0.665234,0.025918,0.910224,-0.979597,0.610582,0.515577,0.055653,0.029394,0.367403,0.276059,0.247405,0.025235,0.048601,0.496631,-0.060994,3
3,0.507289,-1.468973,-0.208351,0.967490,0.318739,-0.599155,0.271468,0.029258,0.281890,0.145417,0.155879,-0.050884,-0.074240,0.066675,-0.163159,3
4,0.676385,0.010505,0.672490,-0.781445,-0.002355,-0.421630,0.204497,0.090873,-0.031931,0.006785,-0.248873,-0.298813,-0.188266,0.556631,0.750617,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,0.635254,0.772877,-0.138328,0.299755,0.504541,0.369877,-0.174631,0.416020,0.084415,0.006464,0.011781,0.022467,0.086665,0.006016,0.007157,0
669,-0.919546,0.013303,0.488923,-0.148207,-0.718547,0.318135,-0.580997,-0.030323,0.213770,0.540197,0.436520,0.159593,0.055932,0.009279,-0.107797,0
670,-1.630891,0.780282,-0.427881,0.073634,-0.629205,0.012114,0.184969,0.197523,-0.022830,0.285207,0.057844,0.113348,-0.004314,0.044037,-0.084071,0
671,0.619389,0.756285,-0.046833,0.374627,-0.176067,-0.316056,-0.207498,-0.263018,0.160468,-0.605431,0.035088,0.410424,0.286489,0.091134,0.022010,2


Se aprecia que el tamaño del DatFrame es de 673 filas y 16 columnas, valores que corresponden a lo esperado puesto que se tienen 673 datos desde el análisis EDA y las 16 columnas corresponden a los componentes principales y la columna objetivo de Performance.

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 673 entries, 0 to 672
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PC1          673 non-null    float64
 1   PC2          673 non-null    float64
 2   PC3          673 non-null    float64
 3   PC4          673 non-null    float64
 4   PC5          673 non-null    float64
 5   PC6          673 non-null    float64
 6   PC7          673 non-null    float64
 7   PC8          673 non-null    float64
 8   PC9          673 non-null    float64
 9   PC10         673 non-null    float64
 10  PC11         673 non-null    float64
 11  PC12         673 non-null    float64
 12  PC13         673 non-null    float64
 13  PC14         673 non-null    float64
 14  PC15         673 non-null    float64
 15  Performance  673 non-null    int64  
dtypes: float64(15), int64(1)
memory usage: 84.3 KB


In [32]:
df.isnull().sum()

PC1            0
PC2            0
PC3            0
PC4            0
PC5            0
PC6            0
PC7            0
PC8            0
PC9            0
PC10           0
PC11           0
PC12           0
PC13           0
PC14           0
PC15           0
Performance    0
dtype: int64

El dataset si cumple con los criterios de integridad y formato requeridos de estar limpios y libres de valores nulos con un formato correcto para su interpretación en el pipeline.

In [29]:
df['Performance'].value_counts(normalize=True)

Performance
1    0.316493
2    0.297177
0    0.234770
3    0.151560
Name: proportion, dtype: float64

In [30]:
df.drop('Performance', axis=1).describe().T
df.drop('Performance', axis=1).var().sort_values().head()

PC15    0.047314
PC14    0.058496
PC13    0.069422
PC12    0.071728
PC11    0.093584
dtype: float64

Se evaluó la varianza de cada una de las 15 variables generadas por el PCA, donde los resultados obtenidos indican que todos los componentes mantienen una varianza positiva y no presentan varianza nula, lo cual indica que el PCA se aplicó correctamente y cada dimensión aporta información útil al modelo. Por lo tanto, el dataset **data-v1.2-features** mantiene una estructura numérica coherente y lista para su uso en el pipeline de modelado a realizar a continuación.

# 2. Model Training and Registering

Este notebook está dedicado al entrenamiento de modelos de machine learning utilizando el dataset obtenido tras las tareas de preprocesamiento. El objetivo es resolver un problema cuya variable obejtivo es una variable categórica ordinal, por lo que se explorarán diferentes algoritmos para identificar el que mejor se adapte a nuestros datos.

En primer lugar, se propondrán tres modelos de clasificación: 
* LightGBM: Algoritmo basado en árboles de decisión optimizado para velocidad y eficiencia, especialmente útil en grandes conjuntos de datos y capaz de manejar variables categóricas y ordinales.
* XGBoost: Implementación avanzada de gradient boosting que destaca por su regularización y manejo eficiente de datos dispersos, logrando alto rendimiento en tareas de clasificación y regresión.
* CatBoosting: Algoritmo de boosting desarrollado por Yandex, diseñado para trabajar de forma nativa con variables categóricas y evitar el overfitting, ofreciendo excelentes resultados en problemas con datos heterogéneos.

Cada uno será entrenado y evaluado además se les dará seguimiento a sus experiment runs asociadas mediante MLflow para asegurar la trazabilidad de los resultados.

## ⚙️ Configuración de MLflow

**Importante**: Configurar MLflow tracking ANTES de cualquier operación para evitar crear `mlruns/` en ubicaciones incorrectas.


In [2]:
# === Configuración MLflow (DEBE SER LO PRIMERO) ===
import os

# Directorio centralizado para experimentos MLflow
MLFLOW_TRACKING_DIR = "../data/mlflow"

# Configurar tracking URI ANTES de cualquier operación MLflow
mlflow.set_tracking_uri(f"file:{MLFLOW_TRACKING_DIR}")

# Crear directorio si no existe
os.makedirs(MLFLOW_TRACKING_DIR, exist_ok=True)

print("="*70)
print("✅ MLflow configurado correctamente")
print("="*70)
print(f"📂 Tracking directory: {MLFLOW_TRACKING_DIR}")
print("💡 Todos los experimentos se guardarán aquí (no en notebooks/mlruns/)")
print("="*70)


✅ MLflow configurado correctamente
📂 Tracking directory: ../data/mlflow
💡 Todos los experimentos se guardarán aquí (no en notebooks/mlruns/)


**función auxiliar para logging**

In [3]:
import logging
import sys
from typing import Optional

def get_logger(name: str = __name__, log_file: Optional[str] = None, level=logging.INFO):
    """
    Create and configure a logger with optional file and console handlers.

    Args:
        name (str): Logger name (usually __name__).
        log_file (str, optional): Path to a file to log messages.
        level (int): Logging level (default: INFO).

    Returns:
        logging.Logger: Configured logger instance.
    """
    logger = logging.getLogger(name)
    logger.setLevel(level)

    if not logger.handlers:  # Prevent duplicate handlers in Jupyter or repeated calls
        formatter = logging.Formatter(
            fmt="[%(asctime)s | %(levelname)s ] %(name)s -> %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S"
        )

        # Console handler
        console_handler = logging.StreamHandler(sys.stdout)
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)

        # Optional file handler
        if log_file:
            file_handler = logging.FileHandler(log_file)
            file_handler.setFormatter(formatter)
            logger.addHandler(file_handler)

    return logger

In [4]:
# definimos el logger
logger = get_logger("mlflow experiments")

# Training Dataset

## Lectura del conjunto de entrenamiento

La variable objetivo, "Performance", es una variable categórica ordinal, lo que implica que sus categorías tienen un orden inherente pero no una distancia cuantificable entre ellas. Para abordar este tipo de problema, emplearemos algoritmos que permiten modelar la ordinalidad de la variable, como LightGBM, XGBoost y CatBoost. 

Es fundamental configurar correctamente los parámetros de estos modelos: 
 * en LightGBM se debe establecer el parámetro `objective` como `"multiclass"` o `"multiclassova"` y, para ordinalidad, considerar el uso de la variante `lgbm.rank` si se desea modelar el orden.
 * en XGBoost, el parámetro `objective` debe ser `"multi:softprob"` para clasificación multiclase; 
 * en CatBoost, se puede utilizar el parámetro `loss_function="MultiClass"` y, para ordinalidad, el modo `"YetiRank"` o `"Ordinal"` si se requiere modelar el orden explícitamente. 
 
 Además, es necesario especificar el número de clases (`num_class` o `classes_count`) y asegurarse de que la codificación de la variable objetivo respete el orden natural de las categorías. Estas configuraciones permiten que los modelos aprovechen la información ordinal y mejoren la capacidad predictiva en este contexto.

In [5]:
# === Leer dataset de features versionado con DVC ===
# Este archivo debe ser la versión data-v1.2-features (después de feature engineering)
DATA_PATH = "../data/processed/student_performance_features.csv"

print("="*70)
print("📂 PASO 3: Cargando dataset de features para entrenamiento")
print("="*70)
print(f"Archivo: {DATA_PATH}")
print("💡 Este archivo está versionado con DVC (data-v1.2-features)")
print("💡 Asegúrate de tener la versión correcta con: dvc pull")
print("="*70 + "\n")

df = pd.read_csv(DATA_PATH)

print(f"✅ Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")
print(f"📊 Features: {df.shape[1]-1} componentes PCA + 1 target (Performance)")
print("="*70)
df.head()

📂 PASO 3: Cargando dataset de features para entrenamiento
Archivo: ../data/processed/student_performance_features.csv
💡 Este archivo está versionado con DVC (data-v1.2-features)
💡 Asegúrate de tener la versión correcta con: dvc pull

✅ Dataset cargado: 673 filas, 16 columnas
📊 Features: 15 componentes PCA + 1 target (Performance)


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,Performance
0,0.803234,-0.597979,-0.285577,-0.770143,-0.017667,0.023372,-0.063243,-0.451023,-0.984059,-0.048342,-0.116927,0.258324,-0.415056,-0.299239,-0.045746,3
1,0.383741,-0.558693,0.664729,1.081237,1.002544,-0.000102,0.260944,0.398450,-0.258088,0.143168,0.352092,0.023940,0.116888,0.069941,-0.081184,3
2,0.665234,0.025918,0.910224,-0.979597,0.610582,0.515577,0.055653,0.029394,0.367403,0.276059,0.247405,0.025235,0.048601,0.496631,-0.060994,3
3,0.507289,-1.468973,-0.208351,0.967490,0.318739,-0.599155,0.271468,0.029258,0.281890,0.145417,0.155879,-0.050884,-0.074240,0.066675,-0.163159,3
4,0.676385,0.010505,0.672490,-0.781445,-0.002355,-0.421630,0.204497,0.090873,-0.031931,0.006785,-0.248873,-0.298813,-0.188266,0.556631,0.750617,3


## Separación en Conjunto de entrenamiento y prueba -estratificado-

In [6]:
target_column = 'Performance'

In [7]:
# X, matriz de caracteristicas
X = df.drop(columns=[target_column])
# y, vairable objetivo
y = df[target_column]

In [8]:
# Separación en conjunto de entrenamient y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=13,
    stratify=y
)

# Model training

El flujo básico de MLflow para la gestión de experimentos de machine learning consta de cuatro pasos fundamentales. 

**1. Definición del experimento:** Antes de iniciar cualquier entrenamiento, se debe crear o seleccionar un experimento en MLflow mediante `mlflow.create_experiment()` o `mlflow.set_experiment()` o `mlflow.get_experiment_by_name(experiment_name)`. Esto permite agrupar y organizar los diferentes intentos de entrenamiento bajo un mismo contexto, facilitando la trazabilidad y comparación de resultados. 

**2. Ejecución de una corrida (run) dentro del experimento:** Cada entrenamiento de modelo se encapsula en una "run", iniciada con `mlflow.start_run()`. Una run representa una ejecución individual, donde se pueden registrar todos los artefactos y métricas asociados. 

**3. Logging de parámetros, métricas y artefactos:** Durante la run, se emplean funciones como `mlflow.log_params()`, `mlflow.log_metrics()` y `mlflow.log_artifact()` para guardar los hiperparámetros utilizados, las métricas de desempeño obtenidas y cualquier archivo relevante (por ejemplo, el modelo entrenado o gráficos de evaluación). Este registro estructurado permite auditar y reproducir los experimentos fácilmente. 

**4. Registro del modelo:** Una vez identificado el mejor modelo, se utiliza `mlflow.register_model()` para almacenarlo en el Model Registry de MLflow. Esto habilita la gestión de versiones, la transición entre estados (staging, production, archived) y el despliegue controlado del modelo, asegurando que el ciclo de vida del modelo esté completamente documentado y gestionado. Este flujo garantiza la reproducibilidad, trazabilidad y gobernanza de los modelos desarrollados.


Es por ello que se ha decidido diseñar la función `evaluate_and_log_model()`. esta función trata de los siguiente:

Claro, aquí tienes una explicación paso a paso de lo que hace la función `evaluate_and_log_model`:

1. **Recibe los argumentos**:  
    - Nombre del modelo, instancia del modelo, datos de entrenamiento y prueba, parámetros, nombre del experimento MLflow y logger.

2. **Busca o crea el experimento en MLflow**:  
    - Si el experimento no existe, lo crea.  
    - Si ya existe, lo reutiliza.  
    - Informa por el logger qué experimento se está usando.

3. **Inicia una corrida (run) en MLflow**:  
    - Cada entrenamiento se encapsula en una run para registrar resultados.

4. **Loguea los parámetros del modelo**:  
    - Si se pasan parámetros, los registra en MLflow y lo informa por el logger.

5. **Entrena el modelo**:  
    - Ajusta el modelo con los datos de entrenamiento (`fit`).  
    - Informa por el logger que el entrenamiento terminó.

6. **Realiza predicciones**:  
    - Predice sobre el conjunto de prueba.  
    - Redondea las predicciones para obtener clases enteras.

7. **Calcula métricas**:  
    - Calcula el RMSE (error cuadrático medio raíz) entre las predicciones y los valores reales.  
    - Calcula el QWK (Cohen’s Kappa ponderado cuadrático) para evaluar la calidad de la clasificación ordinal.

8. **Registra las métricas en MLflow**:  
    - Loguea RMSE y QWK en MLflow.  
    - Informa por el logger los valores obtenidos.

9. **Guarda el modelo en MLflow**:  
    - Registra el modelo entrenado como artefacto en MLflow.  
    - Informa por el logger que el modelo fue guardado.

10. **Devuelve las métricas**:  
     - Retorna RMSE y QWK para su uso posterior.


**Métricas utilizadas: RMSE y QWK**

- **RMSE (Root Mean Squared Error):**  
    El RMSE es una métrica que mide la diferencia promedio entre los valores predichos por el modelo y los valores reales, penalizando más fuertemente los errores grandes. Se calcula como la raíz cuadrada de la media de los errores al cuadrado. En problemas de regresión y clasificación ordinal, el RMSE permite cuantificar qué tan cerca están las predicciones de los valores verdaderos, siendo útil para evaluar modelos que predicen valores continuos o clases ordenadas. Un RMSE bajo indica que el modelo realiza predicciones precisas.

- **QWK (Quadratic Weighted Kappa):**  
    El QWK es una métrica diseñada para evaluar la concordancia entre dos clasificadores (por ejemplo, las predicciones del modelo y las etiquetas reales) en problemas de clasificación ordinal. Considera no solo si la predicción es correcta, sino también cuán lejos está la predicción de la clase verdadera, penalizando más los errores grandes. El QWK toma valores entre -1 y 1, donde 1 indica perfecta concordancia, 0 indica concordancia aleatoria y valores negativos indican peor que aleatorio. Es especialmente relevante en contextos donde las clases tienen un orden natural, como en este caso.

Estas métricas permiten comparar objetivamente el desempeño de los modelos en la predicción de la variable ordinal "Performance", asegurando que tanto la precisión como el respeto por el orden de las categorías sean considerados.

## Función auxiliar para el logging con MLFlow

In [9]:
def evaluate_and_log_model(
    model_name,
    model,
    X_train,
    X_test,
    y_train,
    y_test,
    params=None,
    experiment_name="Default_Experiment",
    logger=None,
    tracking_dir="../data/mlflow"
):
    """
    Train model, log metrics, parameters, and model artifact to MLflow.
    Automatically creates or reuses an MLflow experiment by name.

    Args:
        model_name (str): Name of the model/run.
        model: Untrained model instance (e.g., sklearn model).
        X_train, X_test, y_train, y_test: Training/testing data.
        params (dict, optional): Model parameters to log.
        experiment_name (str): MLflow experiment name.
        logger (logging.Logger, optional): Logger for status messages.
        tracking_dir (str): Local path to store MLflow tracking data.
    """
    # --- Ensure MLflow uses the desired local folder ---
    mlflow.set_tracking_uri(f"file:{tracking_dir}")
    if logger:
        logger.info(f"MLflow tracking directory set to: {tracking_dir}")

    # --- Handle experiment setup ---
    existing_experiment = mlflow.get_experiment_by_name(name=experiment_name)
    if existing_experiment is None:
        experiment_id = mlflow.create_experiment(
            name=experiment_name,
            tags={"owner": "equipo36", "project": "student-performance-prediction"}
        )
        if logger:
            logger.info(f"Created new MLflow experiment: '{experiment_name}' (ID: {experiment_id})")
    else:
        experiment_id = existing_experiment.experiment_id
        if logger:
            logger.info(f"Using existing MLflow experiment: '{experiment_name}' (ID: {experiment_id})")

    # --- Start MLflow run ---
    with mlflow.start_run(experiment_id=experiment_id, run_name=model_name):
        if params:
            mlflow.log_params(params)
            if logger:
                logger.info(f"Logged parameters for {model_name}: {params}")

        # Fit model
        model.fit(X_train, y_train)
        if logger:
            logger.info(f"Model '{model_name}' training complete.")

        # Predict
        y_pred = model.predict(X_test)
        y_pred_class = np.rint(y_pred).astype(int)

        # Compute metrics
        rmse = root_mean_squared_error(y_test, y_pred_class)
        qwk = cohen_kappa_score(y_test, y_pred_class, weights="quadratic")

        # Log metrics
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("quadratic_weighted_kappa", qwk)
        if logger:
            logger.info(f"Metrics logged — RMSE: {rmse:.4f}, QWK: {qwk:.4f}")

        # Log the model
        mlflow.sklearn.log_model(model, artifact_path="model", input_example=X_test.iloc[:5])
        if logger:
            logger.info(f"Model '{model_name}' logged to MLflow under experiment '{experiment_name}'.")

        return rmse, qwk

## MLFlow server

MLflow server es una herramienta que permite gestionar y monitorizar experimentos de machine learning de forma centralizada. Su propósito principal es almacenar los resultados, parámetros, métricas y modelos generados durante el ciclo de vida de los experimentos, facilitando la trazabilidad, comparación y reproducibilidad. Al ejecutar el servidor MLflow, se habilita una interfaz web donde los usuarios pueden visualizar y administrar todos los experimentos registrados en la plataforma.

En la celda siguiente, podemos encontrar código que nos ayuda a levantar el server.

In [10]:
import subprocess, time, os, signal

# Configuration
MLFLOW_HOST = "127.0.0.1"
MLFLOW_PORT = 8080

# Start the MLflow server in background
mlflow_process = subprocess.Popen(
    [
        "mlflow", "server",
        "--host", MLFLOW_HOST,
        "--port", str(MLFLOW_PORT),
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

print(f"✅ MLflow server started at http://{MLFLOW_HOST}:{MLFLOW_PORT}")
time.sleep(3)

✅ MLflow server started at http://127.0.0.1:8080


## LightGBM



### Definición

In [11]:
# Define LightGBM params
params_lgb = {
    'objective': 'regression',    # or 'rmse' — continuous ordinal target
    'metric': 'rmse',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'min_data_in_leaf': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'random_state': 13
}

# Initialize model
model_lgb = lgb.LGBMRegressor(**params_lgb)

### Entrenamiento y logging con mlflow

In [12]:
lgb_rmse, lgb_qwk = evaluate_and_log_model(
    experiment_name="mlflow-student-performance-experiment",
    model_name="LightGBM",
    model=model_lgb,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    params=params_lgb,
    logger=logger
)

[2025-10-30 19:59:55 | INFO ] mlflow experiments -> MLflow tracking directory set to: ../data/mlflow
[2025-10-30 19:59:55 | INFO ] mlflow experiments -> Using existing MLflow experiment: 'mlflow-student-performance-experiment' (ID: 110171930243231981)
[2025-10-30 19:59:55 | INFO ] mlflow experiments -> Logged parameters for LightGBM: {'objective': 'regression', 'metric': 'rmse', 'learning_rate': 0.05, 'num_leaves': 31, 'max_depth': -1, 'min_data_in_leaf': 20, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'random_state': 13}
[2025-10-30 19:59:55 | INFO ] mlflow experiments -> Model 'LightGBM' training complete.
[2025-10-30 19:59:55 | INFO ] mlflow experiments -> Metrics logged — RMSE: 0.8819, QWK: 0.5097


2025/10/30 19:59:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


[2025-10-30 20:00:07 | INFO ] mlflow experiments -> Model 'LightGBM' logged to MLflow under experiment 'mlflow-student-performance-experiment'.


## XGBoost

### Definición

In [13]:
# Definición del modelo
# NOTE: Usamos un diccionario para definir los hiperparámetros del modelo
params_xgb = {
    'objective': 'reg:squarederror',
    'n_estimators': 200,
    'learning_rate': 0.05,
    'max_depth': 6,
    'random_state': 13
}
# Instanciamos el modelo
model_xgb = xgb.XGBRegressor(**params_xgb)

### Entrenamiento y loggeo con mlflow

In [14]:
xgb_rmse, xgb_qwk = evaluate_and_log_model(
    experiment_name="mlflow-student-performance-experiment",
    model_name="XGBoost",
    model=model_xgb,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    params=params_xgb,
    logger=logger
)

[2025-10-30 20:00:07 | INFO ] mlflow experiments -> MLflow tracking directory set to: ../data/mlflow
[2025-10-30 20:00:07 | INFO ] mlflow experiments -> Using existing MLflow experiment: 'mlflow-student-performance-experiment' (ID: 110171930243231981)
[2025-10-30 20:00:07 | INFO ] mlflow experiments -> Logged parameters for XGBoost: {'objective': 'reg:squarederror', 'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 6, 'random_state': 13}
[2025-10-30 20:00:08 | INFO ] mlflow experiments -> Model 'XGBoost' training complete.
[2025-10-30 20:00:08 | INFO ] mlflow experiments -> Metrics logged — RMSE: 0.8477, QWK: 0.5524


2025/10/30 20:00:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


[2025-10-30 20:00:13 | INFO ] mlflow experiments -> Model 'XGBoost' logged to MLflow under experiment 'mlflow-student-performance-experiment'.


## CatBoosting

### Definición del modelo

In [15]:
# Define CatBoost parameters
params_cat = {
    'iterations': 300,
    'learning_rate': 0.05,
    'depth': 6,
    'loss_function': 'RMSE',   # Continuous target, respects ordering
    'random_seed': 42,
    'verbose': 0,
    'od_type': 'Iter',         # Enable early stopping
    'od_wait': 20,
    'train_dir': None          # ⚠️ Evita crear directorio catboost_info/
}

# Initialize model
model_cat = CatBoostRegressor(**params_cat)

### Entrenamiento y loggeo con mlflow

In [16]:
cat_rmse, cat_qwk = evaluate_and_log_model(
    experiment_name="mlflow-student-performance-experiment",
    model_name="CatBoost",
    model=model_cat,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    params=params_cat,
    logger=logger
)

[2025-10-30 20:00:13 | INFO ] mlflow experiments -> MLflow tracking directory set to: ../data/mlflow
[2025-10-30 20:00:13 | INFO ] mlflow experiments -> Using existing MLflow experiment: 'mlflow-student-performance-experiment' (ID: 110171930243231981)
[2025-10-30 20:00:13 | INFO ] mlflow experiments -> Logged parameters for CatBoost: {'iterations': 300, 'learning_rate': 0.05, 'depth': 6, 'loss_function': 'RMSE', 'random_seed': 42, 'verbose': 0, 'od_type': 'Iter', 'od_wait': 20, 'train_dir': None}
[2025-10-30 20:00:13 | INFO ] mlflow experiments -> Model 'CatBoost' training complete.
[2025-10-30 20:00:13 | INFO ] mlflow experiments -> Metrics logged — RMSE: 0.9108, QWK: 0.4491


2025/10/30 20:00:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


[2025-10-30 20:00:18 | INFO ] mlflow experiments -> Model 'CatBoost' logged to MLflow under experiment 'mlflow-student-performance-experiment'.


# Finalmente

Detenemos el servidor de MLFlow

In [17]:
# if 'mlflow_process' in locals() and mlflow_process.poll() is None:
#     os.kill(mlflow_process.pid, signal.SIGTERM)
#     print("🛑 MLflow server stopped.")
# else:
#     print("MLflow server was not running or already stopped.")

## 📦 Versionar Experimentos MLflow con DVC

Una vez que hayas terminado de entrenar y evaluar los modelos, es recomendable versionar los experimentos y artefactos de MLflow con DVC para tener un registro completo del pipeline.


In [18]:
# === Versionar directorio MLflow con DVC ===
MLFLOW_PATH = "../data/mlflow"

print("="*70)
print("✅ PASO 3 COMPLETADO: Modelos entrenados y registrados en MLflow")
print("="*70)
print(f"📂 Directorio MLflow: {MLFLOW_PATH}")
print(f"📊 Experimentos registrados: LightGBM, XGBoost, CatBoost")
print(f"📈 Métricas: RMSE y QWK (Quadratic Weighted Kappa)")
print("\n" + "="*70)
print("📦 SIGUIENTE PASO: Versionar experimentos MLflow con DVC")
print("="*70)
print("\n🚀 Ejecuta este comando en la terminal:\n")
print(f"bash add_to_dvc.sh {MLFLOW_PATH} models-v1.0-baseline 'Baseline models: LightGBM, XGBoost, CatBoost'")
print("\n💡 O si prefieres hacerlo manualmente:")
print(f"dvc add {MLFLOW_PATH}")
print(f"git add {MLFLOW_PATH}.dvc .gitignore")
print('git commit -m "feat: version models - baseline models trained"')
print('git tag -a "models-v1.0-baseline" -m "Baseline models: LightGBM, XGBoost, CatBoost"')
print("dvc push")
print("git push origin $(git rev-parse --abbrev-ref HEAD) --tags")
print("\n" + "="*70)
print("🎉 Pipeline MLOps completo con versionado de datos y modelos!")
print("="*70)
print("\n📌 Resumen de versiones:")
print("  • data-v1.0-raw:      Dataset original sin procesar")
print("  • data-v1.1-cleaned:  Dataset después de EDA y limpieza")
print("  • data-v1.2-features: Dataset con features engineered (PCA)")
print("  • models-v1.0-baseline: Modelos baseline entrenados")
print("\n💡 Para recuperar cualquier versión:")
print("  git checkout <tag-name>")
print("  dvc checkout")
print("="*70)


✅ PASO 3 COMPLETADO: Modelos entrenados y registrados en MLflow
📂 Directorio MLflow: ../data/mlflow
📊 Experimentos registrados: LightGBM, XGBoost, CatBoost
📈 Métricas: RMSE y QWK (Quadratic Weighted Kappa)

📦 SIGUIENTE PASO: Versionar experimentos MLflow con DVC

🚀 Ejecuta este comando en la terminal:

bash add_to_dvc.sh ../data/mlflow models-v1.0-baseline 'Baseline models: LightGBM, XGBoost, CatBoost'

💡 O si prefieres hacerlo manualmente:
dvc add ../data/mlflow
git add ../data/mlflow.dvc .gitignore
git commit -m "feat: version models - baseline models trained"
git tag -a "models-v1.0-baseline" -m "Baseline models: LightGBM, XGBoost, CatBoost"
dvc push
git push origin $(git rev-parse --abbrev-ref HEAD) --tags

🎉 Pipeline MLOps completo con versionado de datos y modelos!

📌 Resumen de versiones:
  • data-v1.0-raw:      Dataset original sin procesar
  • data-v1.1-cleaned:  Dataset después de EDA y limpieza
  • data-v1.2-features: Dataset con features engineered (PCA)
  • models-v1.0-bas